<a href="https://colab.research.google.com/github/7758258Ll/datasets/blob/master/%E2%80%9Csummarize_eval_ipynb%E2%80%9D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Copyright 2021 The Balloon Learning Environment Authors.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

In [8]:
# @title Imports
import collections
import json

from google.colab import files
import matplotlib.pyplot as plt
import pandas as pd

from IPython.display import HTML

In [ ]:
pip install zarr

In [ ]:
pip install gcsfs

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import zarr;
random_state = np.random.RandomState(seed=42);
shape = (4, 21, 21, 10, 9, 2);
chunks = (None,) * 5;
values = np.asarray([-1.0, 1.0], dtype=np.float32);
z = zarr.open('array.zarr', mode='w', shape=shape, chunks=chunks, dtype=np.float32);
z[:] = random_state.choice(values, size=shape)
examples, metadata = tfds.load(name='ble_wind_field', with_info=True)

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/290000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/ble_wind_field/full/1.0.0.incompleteXMPAU4/ble_wind_field-train.tfrecord*.…

Dataset ble_wind_field downloaded and prepared to /root/tensorflow_datasets/ble_wind_field/full/1.0.0. Subsequent calls will reuse this data.


In [ ]:
%ls /root/tensorflow_datasets/ble_wind_field/full/1.0.0

In [3]:
%cd /root/tensorflow_datasets/ble_wind_field/full/1.0.0

/root/tensorflow_datasets/ble_wind_field/full/1.0.0


In [ ]:
!tar czvf download.tar.gz *

In [ ]:
!zip -r folder_name.zip /root/tensorflow_datasets/ble_wind_field/full/1.0.0

In [ ]:
!ls

In [10]:
from google.colab import files
files.download('download.tar.gz')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
FeaturesDict({
    'field': Tensor(shape=(21, 21, 10, 9, 2), dtype=float32),
})

In [ ]:
# @title Upload Data
uploaded_files = files.upload()
dataframes = dict()

for name, data in uploaded_files.items():
  name = name.rsplit('.', maxsplit=1)[0]
  json_data = json.loads(data)
  dataframes[name] = pd.DataFrame.from_dict(json_data)

In [ ]:
# coding=utf-8
# Copyright 2023 The TensorFlow Datasets Authors.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

r"""Historical wind field dataset for the Balloon Learning Environment."""

import concurrent.futures
import dataclasses
import itertools
from typing import Optional

import numpy as np
import tensorflow_datasets.public_api as tfds


@dataclasses.dataclass
class BLEWindFieldConfig(tfds.core.BuilderConfig):
  num_fields: Optional[int] = None


class Builder(tfds.core.GeneratorBasedBuilder):
  """DatasetBuilder for the ble_wind_field dataset."""

  VERSION = tfds.core.Version('1.0.0')
  RELEASE_NOTES = {
      '1.0.0': 'Initial release.',
  }
  BUILDER_CONFIGS = [
      BLEWindFieldConfig(
          name='full',
          description='The entire historical wind field dataset.',
          num_fields=None,
      ),
      BLEWindFieldConfig(
          name='small',
          description='Small sample of 256 fields from the dataset.',
          num_fields=256,
      ),
  ]
  GCS_URL = tfds.core.Path('gs://ble-public/downloads')
  GCS_FILENAME = 'historical_wind_fields.zarr'

  def _info(self) -> tfds.core.DatasetInfo:
    """Dataset metadata."""
    return self.dataset_info_from_configs(
        features=tfds.features.FeaturesDict(
            {
                'field': tfds.features.Tensor(
                    shape=(21, 21, 10, 9, 2),
                    dtype=np.float32,
                    encoding=tfds.features.Encoding.ZLIB,
                ),
            }
        ),
        supervised_keys=None,
        homepage='https://github.com/google/balloon-learning-environment',
    )

  def _split_generators(self, dl_manager: tfds.download.DownloadManager):
    return {'train': self._generate_examples()}

  def _generate_examples(self):
    zarr = tfds.core.lazy_imports.zarr
    gcsfs_store = tfds.core.lazy_imports.gcsfs_store
    zarr_array = zarr.open_array(
        store=gcsfs_store(f'{self.GCS_URL}/{self.GCS_FILENAME}'),
        mode='r',
        synchronizer=zarr.ThreadSynchronizer(),
    )

    # During normal execution we don't expect `self.builder_config.num_fields`
    # to have a value larger than `zarr_array.shape[0]`, but for unit tests the
    # Zarr file used has a very small number of fields, so we take the minimum
    # to avoid trying to load more examples than available.
    num_fields = min(
        self.builder_config.num_fields or zarr_array.shape[0],
        zarr_array.shape[0],
    )

    # Zarr arrays are stored as compressed chunks on disk, and by default
    # read/write operations require to load and decompress entire chunks, even
    # if a single element in the chunk is accessed. The most efficient way of
    # iterating over elements is to load entire chunks and iterate over the
    # chunks' elements. The data can be chunked across all axes, but for
    # simplicity we assume that it's only chunked across the batch axis (which
    # is how the historical wind field array is organized).
    chunk_length = zarr_array.chunks[0]
    num_full_chunks = num_fields // chunk_length
    remainder = num_fields % chunk_length

    slices = (
        slice(i * chunk_length, (i + 1) * chunk_length)
        for i in range(num_full_chunks)
    )
    if remainder:
      start = num_full_chunks * chunk_length
      slices = itertools.chain(slices, (slice(start, start + remainder),))

    keys = itertools.count()
    # We pipeline loading the data and writing it to tfrecords files using a
    # thread that loads chunks asynchronously.
    with concurrent.futures.ThreadPoolExecutor() as executor:
      load_fn = lambda next_slice: zarr_array[next_slice]
      # Keep at most 10 submitted loading tasks to reduce resource consumption.
      futures = {
          executor.submit(load_fn, next_slice)
          for next_slice in itertools.islice(slices, 10)
      }
      while futures:
        done, futures = concurrent.futures.wait(
            futures, return_when=concurrent.futures.FIRST_COMPLETED
        )
        # Top up the submitted loading tasks.
        for next_slice in itertools.islice(slices, len(done)):
          futures.add(executor.submit(load_fn, next_slice))

        # Yield the loaded slices.
        for future in done:
          for field in future.result():
            yield next(keys), {'field': field}


AssertionError: ignored

In [ ]:
LBuilder

AttributeError: ignored

In [ ]:
pip install -q tfds-nightly tensorflow matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 45.6 MB/s eta 0:00:00


In [ ]:
import tensorflow.compat.v2 as tf
import tensorflow_datasets as tfds

# Construct a tf.data.Dataset
ds = tfds.load('ble_wind_field', split='train', shuffle_files=True)



Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/290000 [00:00<?, ? examples/s]

ModuleNotFoundError: ignored

In [ ]:
import tensorflow.compat.v2 as tf
import tensorflow_datasets as tfds
tfds.is_dataset_on_gcs('ble_wind_field')

False

In [ ]:
# Build your input pipeline
ds = ds.shuffle(1024).batch(32).prefetch(tf.data.experimental.AUTOTUNE)
for example in ds.take(1):
  image, label = example["image"], example["label"]

NameError: ignored

In [ ]:
# @title Print Aggregate results
aggregate_data = collections.defaultdict(list)
for name, df in dataframes.items():
  aggregate_data['num episodes'].append(len(df.out_of_power))
  aggregate_data['out of power'].append(df.out_of_power.sum())
  aggregate_data['zeropressure'].append(df.zeropressure.sum())
  aggregate_data['envelope burst'].append(df.envelope_burst.sum())

  finished_runs = df.loc[df.out_of_power == False]
  finished_runs = finished_runs.loc[finished_runs.zeropressure == False]
  finished_runs = finished_runs.loc[finished_runs.envelope_burst == False]
  aggregate_data['mean cumulative reward (finished episodes)'].append(
      finished_runs.cumulative_reward.mean())
  aggregate_data['mean TWR50 (finished episodes)'].append(
      finished_runs.time_within_radius.mean())
  aggregate_data['mean cumulative reward (all episodes)'].append(
      df.cumulative_reward.mean())
  aggregate_data['mean TWR50 (all episodes)'].append(
      df.time_within_radius.mean())

df = pd.DataFrame(aggregate_data)
df.index = dataframes.keys()

# This is a little hacky, but it works 🤷‍♂️
html = df.to_html()
html += "<style>th {max-width: 150px}</style>"
HTML(html)

""
